In [1]:
import numpy as np
from numba import jit
import time 

In [2]:
@jit(nopython=True)
def legendre(n,x):
    leg = 1.
    if n == 0:
        leg = 1
    elif n==1:
        leg = x
    else:
        leg_down1 = x; leg_down2 = 1.
        for i in range(2,n+1):
            leg = (2*i-1)*x*leg_down1/i - (i-1)*leg_down2/i
            leg_down2 = leg_down1
            leg_down1 = leg

    return leg

@jit(nopython=True)
def dlegendre(n,x):
    dleg = 0.
    if n == 0:
        dleg = 0.
    elif n == 1:
        dleg = 1.
    else:
        leg_down1 = x; leg_down2 = 1.
        dleg_down1 = 1.; dleg_down2 = 0.
        for i in range(2,n+1):
            leg = (2*i-1)*x*leg_down1/i - (i-1)*leg_down2/i
            dleg = dleg_down2 + (2*i-1)*leg_down1
            leg_down2 = leg_down1
            leg_down1 = leg
            dleg_down2 = dleg_down1
            dleg_down1 = dleg

    return dleg

@jit(nopython=True)
def gauss_legendre_lobatto(length:int):
    #// define some constants
    tolerance = 4.0 * 1.0e-6
    nnewton_iter = 100
    pi = np.pi
    
    # allocate space
    n = length - 1; #// order of polynomial
    x = np.array([0.0 for i in range(length)])
    w = x * 1.
    if n == 1:
        x[0] = -1.; x[1] = 1.
        w[0] = 1.; w[1] = 1.
    else:
        leg = 0.; dleg = 0.; delta = 0.
        # set end points
        x[0]   = -1.0; x[n] = 1.
        w[0]   =  2./(n*(n+1.)); w[n] =  2./(n*(n+1.))

        for i in range(1,(n+1)//2):
            #// initial guess from an approximate form given by SV Parter (1999)
            x[i] = -np.cos( (i+0.25)*pi/n  - 3/(8*n*pi*(i+0.25)))

            #// newton iteration
            for j in range(nnewton_iter):
                leg = legendre(n+1,x[i]) - legendre(n-1,x[i])
                dleg = dlegendre(n+1,x[i]) - dlegendre(n-1,x[i])
                delta = -leg/dleg
                x[i] += delta
                if (np.abs(delta) <= tolerance * np.abs(x[i]) ): break
            
            x[n-i] = - x[i]
            leg = legendre(n, x[i])
            w[i] = 2./(n*(n+1.)*leg*leg)
            w[n-i] = w[i]

        if n %2 == 0 :
            x[n//2] = 0.
            leg = legendre(n, 0.0)
            w[n//2]  = 2./(n*(n+1.)*leg*leg)
    

    return x,w

@jit(nopython=True)
def lagrange_poly(xi,xctrl):
    nctrl = len(xctrl)
    hprime = np.array([0.0 for i in range(nctrl)])
    h = hprime * 1.0

    #! note: this routine is hit pretty hard by the mesher, optimizing the loops here will be beneficial
    for dgr in range(nctrl):
        prod1 = 1.; prod2 = 1.

        #// lagrangian interpolants
        x0 = xctrl[dgr]
        for i in range(nctrl):
            if i != dgr:
                x = xctrl[i]
                prod1 = prod1*(xi-x)
                prod2 = prod2*(x0-x)

        #//! takes inverse to avoid additional divisions
        #//! (multiplications are cheaper than divisions)
        prod2_inv = 1. / prod2
        h[dgr] = prod1 * prod2_inv

        #// first derivatives
        s = 0.0
        for i in range(nctrl):
            if i != dgr :
                prod3 = 1.0
                for j in range(nctrl):
                    if j != dgr and j != i:
                        prod3 = prod3*(xi-xctrl[j])
                s = s + prod3
        hprime[dgr] = s * prod2_inv
    

    return h,hprime


def init_gll_arrays(NGLLX):

    # gll points and weights
    xgll,wxgll = gauss_legendre_lobatto(NGLLX)

    #// derivative, hprimex[i][j] = l_j'(xi_i)
    hprimex = np.zeros((NGLLX,NGLLX))
    for i in range(NGLLX):
        _,hprimex[i,:] = lagrange_poly(xgll[i],xgll)

    #// hprimex_wgllx[i,j] = hprime_x[i,j] * wx[i] 
    hprimex_wgllx = np.zeros((NGLLX,NGLLX))
    for i in range(NGLLX):
        for j in range(NGLLX):
            hprimex_wgllx[i,j] = hprimex[i,j] * wxgll[i]


    return xgll,wxgll,hprimex,hprimex_wgllx



In [3]:
def create_sem_mesh(zmax,nel,NGLL):
    # connectivity matrix
    ibool = np.zeros((nel,NGLL),dtype=int)
    idx = 0
    for i in range(nel):
        for j in range(NGLL):
            ibool[i,j] = idx 
            idx += 1
        idx -= 1
    
    # jacobians
    jaco = np.zeros((nel))
    jaco[:] = zmax / nel * 0.5 

    return ibool,jaco 
    

In [4]:
@jit(nopython=True)
def assemble_matrix(P,ibool,wgll,hprime,jaco,):
    nspec,NGLL = P.shape
    nglob = np.max(ibool) + 1
    E = np.zeros((nglob,nglob),dtype=float)
    hpT = hprime.transpose()

    for ispec in range(nspec):
        sum_terms=  P[ispec,:] * wgll[:] / jaco[ispec]
        
        for i in range(NGLL):
            ig1 = ibool[ispec,i]
            for j in range(NGLL):
                ig2 = ibool[ispec,j]
                s = np.sum(sum_terms * hpT[i,:] * hpT[j,:])
                E[ig1,ig2] += s 
    
    return E 

@jit(nopython=True)
def assemble_matrix_fd(ibool,wgll,hprime,jaco,x,y,P):
    nspec,NGLL = ibool.shape 
    dm = np.zeros((nspec,NGLL),dtype=float)
    for ispec in range(nspec):
        for i in range(NGLL):
            p = P[ispec,i] * 1. 
            dx = 0.01
            P[ispec,i] = p * (1 + dx)
            E = assemble_matrix(P,ibool,wgll,hprime,jaco)
            s1 = y.dot(E.dot(x))

            P[ispec,i] = p * (1 - dx)
            E = assemble_matrix(P,ibool,wgll,hprime,jaco)
            s2 = y.dot(E.dot(x))

            dm[ispec,i] = (s1 - s2) / (p * dx * 2)

            # recover
            P[ispec,i] = p * 1. 
    
    return dm 
@jit(nopython=True)
def assemble_matrix_deriv(ibool,wgll,hprime,jaco,x,y):
    nspec,NGLL = ibool.shape
    dE = np.zeros((nspec,NGLL))

    # element
    xe = np.zeros((NGLL),dtype=float)
    ye = np.zeros((NGLL),dtype=float)

    for ispec in range(nspec):
        # cache element
        for i in range(NGLL):
            ig = ibool[ispec,i]
            xe[i] = x[ig]
            ye[i] = y[ig]

        # compute 
        sum_terms = wgll[:] / jaco[ispec]
        for k in range(NGLL):
            s1 = np.sum(hprime[k,:] * xe[:])
            s2 = np.sum(hprime[k,:] * ye[:])
            dE[ispec,k] = sum_terms[k] * s1 * s2 
    
    return dE

In [7]:
nspec = 50
NGLL = 20
xgll,wgll,hprime,_ = init_gll_arrays(NGLL)
ibool,jaco = create_sem_mesh(11.5,nspec,NGLL)
nglob = np.max(ibool) + 1

# random seed
np.random.seed(15)
P = np.random.rand(nspec,NGLL)
x = np.random.randn(nglob)
y = np.random.randn(nglob)

tic = time.time()
dm = assemble_matrix_fd(ibool,wgll,hprime,jaco,x,y,P)
toc = time.time()
print('time = ',toc-tic)

tic = time.time()
dE = assemble_matrix_deriv(ibool,wgll,hprime,jaco,x,y)
toc = time.time()
print('time = ',toc-tic)

np.allclose(dE,dm)

time =  2.092771530151367
time =  0.00010395050048828125


True